In [1]:
from pgsql_search.loader import HuggingFaceDatasets

ds = HuggingFaceDatasets("UCSC-VLAA/Recap-COCO-30K")
ds.dataset = ds.dataset.shuffle()
ds = ds.select(list(range(100)))
ds.save_images("../data/images100")
ds = ds.select_columns(["image_filepath", "caption"])
ds.dataset


2024-12-18 17:25:33.946 | INFO     | pgsql_search.loader:__init__:15 - Loading dataset: UCSC-VLAA/Recap-COCO-30K
2024-12-18 17:25:33.953 | INFO     | pgsql_search.loader:save_images:44 - Saving images to folder: ../data/images100


Saving images (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['image_filepath', 'caption'],
    num_rows: 100
})

In [2]:
df = ds.dataset.to_pandas()
df

,image_filepath,caption
0,../data/images100/121430.jpg,A green motorcycle parked on a rugged broken s...
1,../data/images100/162692.jpg,a statue of a lion face attached to a brick wall
2,../data/images100/107228.jpg,Two duffel bags sit next to each other on the ...
3,../data/images100/12552.jpg,A baseball player prepares to hit the baseball...
4,../data/images100/79011.jpg,Three giraffes stand in the grass by a dirt pile.
...,...,...
95,../data/images100/192834.jpg,A bedroom decorated in a futuristic style with...
96,../data/images100/291606.jpg,A woman boarding a bus in a Chinese city.
97,../data/images100/579462.jpg,A woman opening a suitcase on the bed
98,../data/images100/423588.jpg,Four persons skiing on snow clad mountains and...


In [3]:
from pgsql_search.database import PostgreSQLDatabase, ColumnType

PostgreSQLDatabase.create_database("my_database")

2024-12-18 17:25:34.405 | INFO     | pgsql_search.database:create_database:300 - Database 'my_database' already exists


In [4]:
with PostgreSQLDatabase("my_database") as db:
    # First, create the table with just an ID column
    db.initialize_table("image_metadata")
    db.add_column("image_filepath", ColumnType.TEXT, nullable=False)
    db.add_column("caption", ColumnType.TEXT, nullable=True)


    db.insert_dataframe(df)

2024-12-18 17:25:34.454 | INFO     | pgsql_search.database:connect:86 - Connected to database
2024-12-18 17:25:34.458 | INFO     | pgsql_search.database:setup_pgvector_extension:103 - pgvector extension initialized
2024-12-18 17:25:34.461 | INFO     | pgsql_search.database:initialize_table:125 - Initialized table 'image_metadata' with ID column 'id'
2024-12-18 17:25:34.463 | INFO     | pgsql_search.database:add_columns:174 - Added 1 new columns to image_metadata
2024-12-18 17:25:34.464 | INFO     | pgsql_search.database:add_columns:174 - Added 1 new columns to image_metadata
2024-12-18 17:25:34.472 | INFO     | pgsql_search.database:insert_dataframe:226 - Inserted batch 1 (100/100 rows)
2024-12-18 17:25:34.473 | INFO     | pgsql_search.database:insert_dataframe:230 - Successfully inserted 100 rows into image_metadata
2024-12-18 17:25:34.474 | INFO     | pgsql_search.database:disconnect:95 - Disconnected from database
